<a href="https://colab.research.google.com/github/mguelgui/GoogleTrendsScraper/blob/main/TrendsScraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# The request failed: Google returned a response with code 429 :(

# pandas ist eine wichtige Python-Bibliothek zum Arbeiten mit Daten/Tabellen (Excel, CSV)
# Die pytrends-Bibliothek enthält Werkzeuge, um Google Trends Daten automatisch abzufragen; sie liefert die Ergebnisse in einem pandas dataframe (df)
# Hier werden die beiden Bibliotheken gleichzeitig im Terminal installiert
!pip install pytrends pandas

from pytrends.request import TrendReq     # Importiert das Hauptmodul von pytrends, um Anfragen an Google Trends zu senden
import pandas as pd   # Für die Datenmanipulation (Umwandlung von Google-Daten in dfs)
import time           # ! Wichtig: simuliert später eine kleine menschliche Pause und verringert damit das Risiko, als bot erkannt zu werden. Außerdem minimiert es das Risiko, leere Dateien/Errors zu bekommen -> zu viele zu schnelle Anfragen verbietet Trends
import random         # Zum time setzen
import os             # Für Datei- und Ordneroperationen (z. B. Ordner erstellen)

pytrends = TrendReq(hl='de', tz=360)      # Sprache: Deutsch, Zeitzone: 360 = UTC+6; generic Voreinstellung

#############################################
### ACHTUNG HIER IMMER DAS DATUM ANPASSEN ###
#############################################
output_dir = "/content/20250616"          # Ordner nach Datumsprinzip benennen: JahrMonatTag
os.makedirs(output_dir, exist_ok=True)    # Erstellt den Ordner automatisch (nicht mehr manuell nötig)




#########################
### EINZELNE BEGRIFFE ###
#########################

### VORBEREITUNG ###

# Deutsche Begriffe für Deutschland, die bei Trends abgefragt werden
german_terms = ['B1', 'B2', 'B3', 'B4', 'B5', 'B6']

# Englische Begriffe für jeweils Indien, Kalifornien, New York, Texas, Vereinigtes Königreich
english_terms = [
    'b1', 'b2', 'b3', 'b4',
    'b5', 'b6', 'b7', 'b8',
    'b9', 'b10', 'b11', 'b12'
]

# Zuordnung der Ländercodes; Werte sind geo-Parameter, wie Google sie erwartet
english_lands = {
    'IN': 'IN',       # Indien
    'CA': 'US-CA',    # Kalifornien
    'NY': 'US-NY',    # New York
    'TA': 'US-TX',    # Texas
    'VK': 'GB'        # Vereinigtes Königreich
}

# Zeitraum: letzte 7 Tage
timeframe = 'now 7-d'


### DEUTSCHE BEGRIFFE ABFRAGEN ###

for term in german_terms:   # Iteriert über alle Begriffe in dem DE-Vektor
    try:
        pytrends.build_payload([term], cat=0, timeframe=timeframe, geo='DE') # Scraper-Funktion aus pytrends
        data = pytrends.interest_over_time() # pytrends-Funktion; holt Trenddaten aus Google Trends
        if not data.empty:    # pandas-Funktion prüft ob df leer ist; Mögliches Problem bei zu schnellen Anfragen; falls nicht...
            data.drop(columns=['isPartial'], inplace=True, errors='ignore') # 1.
            filename = f"{term}_DE.csv"
            filepath = os.path.join(output_dir, filename)
            data.to_csv(filepath)
            print(f"Gespeichert: {filename}")
        time.sleep(random.uniform(40, 90))  # Error Vermeidung
    except Exception as e:
        print(f"Fehler bei {term}_DE: {e}")


### ENGLISCHE BEGRIFFE ABFRAGEN ###

for term in english_terms:
    for suffix, country_code in english_lands.items():
        try:
            pytrends.build_payload([term], cat=0, timeframe=timeframe, geo=country_code)
            data = pytrends.interest_over_time()
            if not data.empty:
                data.drop(columns=['isPartial'], inplace=True, errors='ignore')
                filename = f"{term}_{suffix}.csv"
                filepath = os.path.join(output_dir, filename)
                data.to_csv(filepath)
                print(f"Gespeichert: {filename}")
            time.sleep(random.uniform(40, 90))
        except Exception as e:
            print(f"Fehler bei {term}_{suffix}: {e}")



######################
### DIE VERGLEICHE ###
######################

# Python Dictionary aus Vergleichbegriffen
vergleichsgruppen_de = {
    "5_Ekel_DE": ["B1", "B2", "B3", "B4", "B6"],
    "5_Überraschung_DE": ["B1", "B2", "B3", "B4", "B5"]
}

vergleichsgruppen_eng = {
    "5_disgust": ["b1", "b2", "b3", "b4", "b6"],
    "5_surprise": ["b1", "b2", "b3", "b4", "b5"]
}

# Ortsübersicht für englische Wörter
englische_laender = {
    "CA": "US-CA",
    "TX": "US-TX",
    "NY": "US-NY",
    "VK": "GB",
    "IN": "IN"
}

# Hauptfunktion:
def fetch_and_save_comparison(keywords, geo_code, filename):
    try:
        pytrends.build_payload(keywords, timeframe="now 7-d", geo=geo_code) # Anfrage an Trends mit den Parametern keywords, letzten 7 Tage und Ort
        df = pytrends.interest_over_time()
        if not df.empty:
            df = df.reset_index()
            df.to_csv(os.path.join(output_dir, filename), index=False)
            print(f"Gespeichert: {filename}")
        else:
            print(f"Keine Daten für: {filename}")
    except Exception as e:
        print(f"Fehler bei {filename}: {e}") # Falls etwas passiert, zB Trends gibt wieder Error an, kommt die Meldung und das Programm läuft weiter

# Durchlaufe alle deutschsprachigen Gruppen
for name, keywords in vergleichsgruppen_de.items():
    fetch_and_save_comparison(keywords, "DE", f"{name}.csv")
    time.sleep(random.uniform(40, 90))   # Error-Vermeidung

# Durchlaufe alle englischsprachigen Gruppen und Länder
for basis_name, keywords in vergleichsgruppen_eng.items():
    for kurzname, geo in englische_laender.items():
        dateiname = f"{basis_name}_{kurzname}.csv"
        fetch_and_save_comparison(keywords, geo, dateiname)
        time.sleep(random.uniform(40, 90))   # Error-Vermeidung

print("Alle Vergleiche abgeschlossen.")